# 🎟️ Get Your Guide

- **Setup:** importa Selenium crea el `driver` de Chrome con esperas.
- **Abrir & UX:** abre la URL objetivo; acepta **cookies**.
- **Preparación:** ejecuta **scroll** / **paginación** para cargar más elementos.
- **Scraping:** recorre tarjetas de **reseñas** y extrae campos clave (puntuación, fecha, texto...).
- **Guardado:** exporta los resultados a **CSV** y **cierra** el navegador.

## 🧰 Librerías e importaciones

In [1]:
import nest_asyncio
import asyncio
from playwright.async_api import async_playwright
import csv
import re

nest_asyncio.apply()

ModuleNotFoundError: No module named 'playwright'

## 🧩 Helpers / utilidades

In [ ]:
def limpiar_num_comentarios(texto):
    if texto:
        num = re.sub(r'[^\d]', '', texto)
        return int(num) if num.isdigit() else None
    return None

async def main():
    url = "https://www.getyourguide.es/valencia-l49/"

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        context = await browser.new_context()
        await context.clear_

## 🍪✅ UX: aceptar cookies (si aparece)

In [ ]:
        await context.clear_cookies()
        page = await context.new_page()
        page.set_default_timeout(30000)

        await page.goto(url)

        # Intentar cerrar banner de
        # Intentar cerrar banner de cookies
        try:
            await page.click('button:has-text("Aceptar")', timeout=5000)
            print(" Botón de
            print(" Botón de cookies aceptado.")
        except:
            print(" No se mostró banner de
            print(" No se mostró banner de cookies.")

        # Esperar a que cargue al menos una tarjeta
        try:
            await page.wait_for_selector('a.vertical-activity-card__container', timeout=30000)
            print(" Tarjetas detectadas en la página.")
        except:
            print(" No se detectaron tarjetas en el tiempo esperado.")
            await browser.close()
            return

        # Pulsar repetidamente el botón "Ver más" mientras esté activo
        while True:
            try:
                boton_ver_mas = await page.query_selector('button:has-text("Ver más")')
                if not boton_ver_mas:
                    print(" No se encontró botón 'Ver más', finalizando carga.")
                    break

                is_disabled = await boton_ver_mas.get_property('disabled')
                if await is_disabled.json_value():
                    print(" Botón 'Ver más' está deshabilitado, finalizando carga.")
                    break

                print(" Pulsando botón 'Ver más' para

## ⬇️↻ Preparación: scroll y paginación

In [ ]:
                print(" Pulsando botón 'Ver más' para cargar más resultados...")
                await boton_ver_mas.click()
                await page.wait_for_timeout(3000)  # Esperar a que carguen resultados nuevos

            except Exception as e:
                print(f" Error al intentar pulsar 'Ver más': {e}")
                break

        # Hacer
        # Hacer scroll para forzar carga adicional (opcional)
        tarjetas_previas = 0
        max_
        max_scrolls = 50
        for i in range(max_
        for i in range(max_scrolls):
            await page.mouse.wheel(0, 3000)
            await page.wait_for_timeout(2500)
            tarjetas = await page.query_selector_all('a.vertical-activity-card__container')
            print(f" Scroll {i+1}: tarjetas encontradas = {len(tarjetas)}")
            if len(tarjetas) == tarjetas_previas:
                break
            tarjetas_previas = len(tarjetas)

        # Extraer datos de las tarjetas
        experiencias = []
        for tarjeta in tarjetas:
            try:
                nombre_el = await tarjeta.query_selector('h3[data-test-id="activity-card-title"] span')
                nombre = await nombre_el.inner_text() if nombre_el else None
            except:
                nombre = None

            try:
                puntuacion_el = await tarjeta.query_selector('div.c-activity-rating__rating')
                puntuacion = await puntuacion_el.inner_text() if puntuacion_el else None
            except:
                puntuacion = None

            try:
                comentarios_el = await tarjeta.query_selector('div.c-activity-rating__label')
                num_comentarios_texto = await comentarios_el.inner_text() if comentarios_el else None
                num_comentarios = limpiar_num_comentarios(num_comentarios_texto)
            except:
                num_comentarios = None

            try:
                precio_el = await tarjeta.query_selector('span.activity-price__text-price')
                precio = await precio_el.inner_text() if precio_el else None
            except:
                precio = None

            try:
                imagen_el = await tarjeta.query_selector('img.c-image__img')
                imagen = await imagen_el.get_attribute('src') if imagen_el else None
            except:
                imagen = None

            try:
                enlace = await tarjeta.get_attribute('href')
                if enlace:
                    enlace = "https://www.getyourguide.es" + enlace
            except:
                enlace = None

            experiencias.append({
                "nombre": nombre,
                "puntuacion": puntuacion,
                "num_comentarios": num_comentarios,
                "precio": precio,
                "imagen": imagen,
                "enlace": enlace
            })

        # Guardar CSV correctamente
        archivo = "experiencias_getyourguide_Valencia.csv"
        with open(archivo, mode='w', newline='', encoding='utf-8') as f:
            campos = ["nombre", "puntuacion", "num_comentarios", "precio", "imagen", "enlace"]
            writer =

## 💾 Guardado de resultados

In [ ]:
            writer = csv.DictWriter(f, fieldnames=campos)
            writer.writeheader()
            writer.
            writer.writerows(experiencias)

        print(f" Scraping completado. {len(experiencias)} experiencias guardadas en {archivo}")
        await browser.close()

asyncio.run(main())